In [7]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import data, io, filters, color, exposure
from skimage.transform import rotate
import re
import os
import re
import shutil

In [8]:
image_path='/Volumes/TOSHIBA EXT/Projects/Datasets/dataset_for_3D_reconstruction/image'


In [9]:
def copy_files_by_pattern(directory_path, destination_path, regex_pattern):
    """
    Copies files from the source directory to the destination directory based on a regex pattern.

    Args:
    directory_path (str): Path to the directory containing the files.
    destination_path (str): Directory where the copies will be stored.
    regex_pattern (str): Regular expression pattern to match filenames.

    Returns:
    None
    """
    # Compile the regex pattern
    pattern = re.compile(regex_pattern)

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_path, exist_ok=True)

    # Initialize a counter for copied files
    copied_files_count = 0

    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the filename matches the pattern
        if pattern.match(filename):
            # Path of the current file
            current_file_path = os.path.join(directory_path, filename)
            # Path where the file will be copied
            destination_file_path = os.path.join(destination_path, filename)
            # Copy the file
            shutil.copy2(current_file_path, destination_file_path)
            copied_files_count += 1
            print(f'Copied: {filename}')

    # Print the result
    print(f'All matching files have been copied to {destination_path}. Total files copied: {copied_files_count}')




In [10]:
cell_types = [
    "spherocyte",
    "echinocyte",
    "keratocytes",
    "multilobate_cells",
    "cell_clusters",
    "knizocytes",
    "multilobate_cells",
    "acanthocytes",
    "stomatocyte"
]

In [11]:
for cell_type in cell_types:
    directory_path = image_path
    destination_path = image_path + '/' + cell_type
    regex_pattern = rf'.*{cell_type}.*\.tif$'

    copy_files_by_pattern(directory_path, destination_path, regex_pattern)

Copied: -1.00_spherocyte000043.tif
Copied: -1.00_spherocyte000044.tif
Copied: -1.00_spherocyte000045.tif
Copied: -1.00_spherocyte000046.tif
Copied: -1.00_spherocyte000047.tif
Copied: -1.00_spherocyte000048.tif
Copied: -1.00_spherocyte000049.tif
Copied: -1.00_spherocyte000050.tif
Copied: -1.00_spherocyte000051.tif
Copied: -1.00_spherocyte000052.tif
Copied: -1.00_spherocyte000053.tif
Copied: -1.00_spherocyte000054.tif
Copied: -1.00_spherocyte000055.tif
Copied: -1.00_spherocyte000056.tif
Copied: -1.00_spherocyte000057.tif
Copied: -1.00_spherocyte000058.tif
Copied: -1.00_spherocyte000059.tif
Copied: -1.00_spherocyte000060.tif
Copied: -1.00_spherocyte000061.tif
Copied: -1.00_spherocyte000062.tif
Copied: -1.00_spherocyte000063.tif
Copied: -1.00_spherocyte000064.tif
Copied: -1.00_spherocyte000065.tif
Copied: -1.00_spherocyte000066.tif
Copied: -1.00_spherocyte000067.tif
Copied: -1.00_spherocyte000068.tif
Copied: -1.00_spherocyte000069.tif
Copied: -1.00_spherocyte000070.tif
Copied: -1.00_sphero

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential

class Patches(Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.pos_embed = Embedding(input_dim=num_patches, output_dim=projection_dim)

    def build(self, input_shape):
        self.dense_proj = Dense(self.projection_dim, use_bias=False)

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        x = self.dense_proj(patch) + self.pos_embed(positions)
        return x

def transformer_block(x, projection_dim, num_heads, mlp_dim, dropout=0.1):
    # Multi-head self-attention
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=projection_dim, dropout=dropout)(x, x)
    # Skip connection 1
    x = tf.keras.layers.Add()([attention_output, x])
    # LayerNorm 1
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    # MLP
    x_intermediate = tf.keras.layers.Dense(mlp_dim, activation=tf.nn.gelu)(x)
    x_output = tf.keras.layers.Dense(projection_dim)(x_intermediate)
    # Skip connection 2
    x = tf.keras.layers.Add()([x_output, x])
    # LayerNorm 2
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x


/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/compvision/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [21]:
def build_vit_classifier(num_classes, image_size=64, patch_size=8, projection_dim=128, num_heads=8, transformer_layers=6, mlp_dim=256):
    inputs = tf.keras.layers.Input(shape=(image_size, image_size, 3))
    patches = Patches(patch_size)(inputs)
    num_patches = (image_size // patch_size) ** 2
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    x = encoded_patches
    for _ in range(transformer_layers):
        x = transformer_block(x, projection_dim, num_heads, mlp_dim, dropout=0.1)

    # Classifier head
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(mlp_dim, activation='relu')(x)  # First Dense layer
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(mlp_dim // 2, activation='relu')(x)  # Second Dense layer
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set paths to your data directories

data_dir = image_path+'/Class'


datagen = ImageDataGenerator(
    rescale=1./255,    # Normalize pixel values
    validation_split=0.2  # Reserve 20% for validation
)

# Setup the train generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # Load images at 64x64
    batch_size=32,
    class_mode='sparse',  # Ensure this matches the loss function
    subset='training',  # Set as training data
    shuffle=True
)

# Setup the validation generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # Load images at 64x64
    batch_size=32,
    class_mode='sparse',  # Ensure this matches the loss function
    subset='validation',  # Set as validation data
    shuffle=True
)

Found 523 images belonging to 8 classes.


Found 126 images belonging to 8 classes.


In [23]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_vit_classifier(num_classes=8)  # Assuming 8 classes
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=50,  # You can set a higher epoch if early stopping is used
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


Epoch 1/50


/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/compvision/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.2485 - loss: 2.2738 - val_accuracy: 0.4603 - val_loss: 1.6888
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.3985 - loss: 1.9180 - val_accuracy: 0.4603 - val_loss: 1.6792
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.4210 - loss: 1.8101 - val_accuracy: 0.4603 - val_loss: 1.6614
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.4559 - loss: 1.6929 - val_accuracy: 0.4603 - val_loss: 1.6816
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4070 - loss: 1.7855 - val_accuracy: 0.4603 - val_loss: 1.6553
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.4229 - loss: 1.7830 - val_accuracy: 0.4603 - val_loss: 1.6850


In [24]:
# !pip install keras-tuner
from kerastuner import RandomSearch
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    num_layers = hp.Int('num_layers', min_value=2, max_value=6, step=1)
    num_heads = hp.Int('num_heads', min_value=4, max_value=12, step=2)
    d_model = hp.Int('d_model', min_value=64, max_value=256, step=64)
    mlp_dim = hp.Int('mlp_dim', min_value=128, max_value=512, step=64)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    
    model = build_vit_classifier(
        num_classes=8, 
        image_size=64, 
        patch_size=8, 
        projection_dim=d_model, 
        num_heads=num_heads, 
        transformer_layers=num_layers, 
        mlp_dim=mlp_dim
    )
    lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=Adam(learning_rate=lr), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of variations
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'
)

tuner.search(train_generator, epochs=10, validation_data=validation_generator)


Trial 10 Complete [00h 07m 13s]
val_accuracy: 0.5952380895614624

Best val_accuracy So Far: 0.5952380895614624
Total elapsed time: 00h 53m 38s


In [29]:
from tensorflow.keras.models import load_model
import json
model_path='/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/Final Project/AdvancedCompVisionProject/my_dir/hparam_tuning/trial_09/checkpoint.weights.h5'

# Reconstruct the model from the saved build configuration
with open('/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/Final Project/AdvancedCompVisionProject/my_dir/hparam_tuning/trial_09/trial.json', 'r') as json_file:
    build_config = json.load(json_file)

# Assuming `build_model` is a function that can take the hyperparameters from
# the build configuration and return a compiled Keras model:
hp = build_config['hyperparameters']  # this may vary depending on the exact structure of your build_config.json
model = build_model(hp)

# Now load the weights
model.load_weights(model_path)

# You can now use model to evaluate, make predictions, etc.


AttributeError: 'dict' object has no attribute 'Int'

In [25]:
from tensorflow.keras.models import load_model

model_path='/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/Final Project/AdvancedCompVisionProject/my_dir/hparam_tuning/trial_09/checkpoint.weights.h5'
# Load the model
model = load_model(model_path)

# Use the model for predictions
predictions = model.predict(validation_generator)


ValueError: No model config found in the file at /Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/Final Project/AdvancedCompVisionProject/my_dir/hparam_tuning/trial_09/checkpoint.weights.h5.

In [33]:
from tensorflow.keras.optimizers import Adam

# Assuming the build_vit_classifier function is defined elsewhere and it is designed
# to build the model according to the given hyperparameters
model = build_vit_classifier(
    num_classes=8,  # Number of classes
    image_size=64,  # Image size
    patch_size=8,  # Patch size
    projection_dim=192,  # d_model equivalent from JSON
    num_heads=10,  # num_heads from JSON
    transformer_layers=5,  # num_layers from JSON
    mlp_dim=384  # mlp_dim from JSON
)

# Set the learning rate as found in the JSON
learning_rate = 7.662902023326896e-05

# Compile the model with the optimizer and learning rate from the hyperparameters
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_path='/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/Final Project/AdvancedCompVisionProject/my_dir/hparam_tuning/trial_09/checkpoint.weights.h5'

# Load the weights
model.load_weights(model_path)

# Now, the model is ready to be used for evaluation or further training


/Users/kanumadhok/Desktop/Desktop Kanu/UChicago/Class/AdvancedComputerVisionDeepLearning/compvision/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 178 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [34]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 898ms/step - accuracy: 0.6058 - loss: 1.4137
Validation accuracy: 59.52%


In [36]:
predictions = model.predict(validation_generator)
predictions

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 901ms/step


array([[0.1622264 , 0.10125488, 0.24668248, ..., 0.07036846, 0.19767684,
        0.07806119],
       [0.32759884, 0.06651262, 0.19850148, ..., 0.06150077, 0.1627068 ,
        0.04101174],
       [0.10981067, 0.11962286, 0.37509245, ..., 0.05189702, 0.13290685,
        0.0852427 ],
       ...,
       [0.04098357, 0.15492082, 0.60078555, ..., 0.02791126, 0.03793916,
        0.08649072],
       [0.0150673 , 0.1623144 , 0.71190464, ..., 0.01710396, 0.01143766,
        0.06273289],
       [0.11631525, 0.10929331, 0.41672182, ..., 0.04564495, 0.11654524,
        0.07627453]], dtype=float32)

In [37]:
import numpy as np

class_predictions = np.argmax(predictions, axis=1)
class_predictions


array([2, 0, 2, 0, 2, 2, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 6, 2, 6, 2, 2, 2, 2, 2, 2, 6, 2, 6, 2, 2, 2, 2, 2, 2,
       2, 6, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 6, 2, 2, 2,
       2, 6, 2, 6, 2, 2, 2, 0, 2, 2, 2, 2, 2, 6, 2, 2, 2, 0, 6, 6, 2, 6,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [40]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming the build_vit_classifier function is defined elsewhere and it is designed
# to build the model according to the given hyperparameters
# Rebuild the model with the same hyperparameters that gave the best results
# but with increased complexity
model = build_vit_classifier(
    num_classes=8,
    image_size=64,
    patch_size=8,
    projection_dim=250,  # Consider increasing this value
    num_heads=10,
    transformer_layers=10,  # Consider adding more layers
    mlp_dim=450  # Consider increasing this value
)


# Set the learning rate as found in the JSON
learning_rate = 7.662902023326896e-05

# Compile the model with the optimizer and learning rate from the hyperparameters
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Set up model checkpointing
model_checkpoint = ModelCheckpoint(
    'best_model_vit_addedcomplex.keras',  # Path where to save the model
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    save_freq='epoch'
)


# Continue training
history = model.fit(
    train_generator,
    epochs=100,  # More epochs than before
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)



Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2145 - loss: 2.2477
Epoch 1: val_accuracy improved from -inf to 0.46032, saving model to best_model_vit_addedcomplex.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 155s 7s/step - accuracy: 0.2175 - loss: 2.2415 - val_accuracy: 0.4603 - val_loss: 1.7265
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2611 - loss: 1.9924
Epoch 2: val_accuracy did not improve from 0.46032
17/17 ━━━━━━━━━━━━━━━━━━━━ 112s 7s/step - accuracy: 0.2645 - loss: 1.9899 - val_accuracy: 0.4603 - val_loss: 1.6468
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3463 - loss: 1.8769
Epoch 3: val_accuracy did not improve from 0.46032
17/17 ━━━━━━━━━━━━━━━━━━━━ 112s 7s/step - accuracy: 0.3474 - loss: 1.8760 - val_accuracy: 0.4603 - val_loss: 1.6406
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.3721 - loss: 1.8315
Epoch 4: val_accuracy did not improve from 0.46032
17/17 ━━━━━━━━━━━━━━━━━━━━ 142s 8s/step - accuracy: 0.3734 -